In [ ]:
"""连接OpenAIAPI"""
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
"""忽略警告"""
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
"""RAG实践2：对话式交互和多步骤检索（没学完。。。）"""
# 选择聊天模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# 选择嵌入模型
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 选择向量存储
from langchain_chroma import Chroma
vector_store = Chroma(embedding_function=embeddings)

# RAG第一部分
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# MessagesState 可以用来表示聊天机器人中各个对话节点的状态。
# StateGraph 用来构建对话流图，每个节点表示用户与系统的某一对话状态。
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)